# Read results processed by MintPy

**Copyright (c) 2020, Lei Yuan**

## all functions

In [ ]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
from xml.dom.minidom import parse
import zipfile
import datetime
from osgeo import gdal, osr
%matplotlib inline


def read_h5(fname, label):
    with h5py.File(fname, 'r') as f:
        atr = dict(f.attrs)
        data = np.asarray(f[(label)])
    return data, atr


def read_vel(vel_file, mask_file=None, ds_factor=1, out_vel_file=None):
    vel, atr = read_h5(vel_file, 'velocity')

    lon = float(atr['X_FIRST'])
    lat = float(atr['Y_FIRST'])

    lon_step = float(atr['X_STEP'])
    lat_step = float(atr['Y_STEP'])

    lon_tmp = (np.linspace(lon, lon + lon_step * vel.shape[1], vel.shape[1]))
    lat_tmp = (np.linspace(lat, lat + lat_step * vel.shape[0], vel.shape[0]))
    lons, lats = np.meshgrid(lon_tmp, lat_tmp)

    vel = vel[::ds_factor, ::ds_factor]
    lons = lons[::ds_factor, ::ds_factor]
    lats = lats[::ds_factor, ::ds_factor]

    if mask_file:
        mask, _ = read_h5(mask_file, 'mask')
        mask = mask[::ds_factor, ::ds_factor]

        lons = lons[mask].reshape((-1, 1))
        lats = lats[mask].reshape((-1, 1))
        vel = vel[mask].reshape((-1, 1))
    else:
        lons = lons.reshape((-1, 1))
        lats = lats.reshape((-1, 1))
        vel = vel.reshape((-1, 1))

    num = np.arange(vel.shape[0]).reshape((-1, 1))

    vel *= 1000

    print('max velocity : ', np.max(vel))
    print('min velocity : ', np.min(vel))
    print('number of points : ', vel.shape[0])

    out_data = np.hstack((num, lons, lats, vel))

    if out_vel_file:
        print('writing data to {}'.format(out_vel_file))
        np.savetxt(out_vel_file, out_data, fmt='%4f')
        print('done.')


def read_vel_ts(ts_file, vel_file, mask_file, ds_factor=1, out_vel_file=None, out_ts_file=None):
    mask, _ = read_h5(mask_file, 'mask')
    mask = np.asarray(mask)

    vel, _ = read_h5(vel_file, 'velocity')
    vel = np.asarray(vel) * 1000

    date, _ = read_h5(ts_file, 'date')
    date = date.astype(np.int64)
    ts, atr = read_h5(ts_file, 'timeseries')
    ts = np.asarray(ts)
    ts = ts.reshape((date.shape[0], vel.shape[0], vel.shape[1])) * 1000
    ts = ts[:, ::ds_factor, ::ds_factor]
    ts = ts.reshape((date.shape[0], -1, 1))

    lon = float(atr['X_FIRST'])
    lon_step = float(atr['X_STEP'])

    lat = float(atr['Y_FIRST'])
    lat_step = float(atr['Y_STEP'])

    lon_tmp = (np.linspace(lon, lon + lon_step * vel.shape[1], vel.shape[1]))
    lat_tmp = (np.linspace(lat, lat + lat_step * vel.shape[0], vel.shape[0]))

    lons, lats = np.meshgrid(lon_tmp, lat_tmp)

    lons = lons[::ds_factor, ::ds_factor]
    lats = lats[::ds_factor, ::ds_factor]
    mask = mask[::ds_factor, ::ds_factor]
    vel = vel[::ds_factor, ::ds_factor]

    lons = lons.reshape((-1, 1))
    lats = lats.reshape((-1, 1))
    vels = vel.reshape((-1, 1))
    mask = mask.reshape((-1, 1))

    lons = lons[mask].reshape((-1, 1))
    lats = lats[mask].reshape((-1, 1))
    vels = vels[mask].reshape((-1, 1))
    num = np.arange(lons.shape[0]).reshape((-1, 1))

    print('number of points : ', lons.shape[0])
    print('max velocity : ', np.max(vels))
    print('min velocity : ', np.min(vels))
    out_vel = np.hstack((num, lons, lats, vels))

    out_ts1 = out_vel
    for i in range(ts.shape[0]):
        data = ts[i]
        out_ts1 = np.hstack((out_ts1, data[mask].reshape((-1, 1))))

    tmp = out_ts1[:, 4:]
    tmp = tmp - tmp[:, 0].reshape((-1, 1))
    tmp = np.hstack((out_vel, tmp))

    out_ts1 = tmp
    print('max cumulative displacement: ', np.max(out_ts1[:, -1]))
    print('min cumulative displacement: ', np.min(out_ts1[:, -1]))

    first_line = np.asarray([[-1, -1, -1, -1]])
    first_line = np.hstack((first_line, date.reshape((1, -1))))
    out_ts = np.vstack((first_line, out_ts1))
    out_vel = out_ts1[0:, 0:4]

    if out_ts_file:
        print('writing data to {}'.format(out_ts_file))
        np.savetxt(out_ts_file, out_ts, fmt='%4f')
        print('done.')

    if out_vel_file:
        print('writing data to {}'.format(out_vel_file))
        np.savetxt(out_vel_file, out_vel, fmt='%4f')
        print('done.')

    return out_ts, date


def vel2geotiff(vel_file, out_file, mask_file=None):
    vel, atr = read_h5(vel_file, 'velocity')
    vel *= 1000

    min_lon = float(atr['X_FIRST'])
    max_lat = float(atr['Y_FIRST'])

    lon_step = float(atr['X_STEP'])
    lat_step = float(atr['Y_STEP'])

    width, nlines = vel.shape[1], vel.shape[0]

    if mask_file:
        mask, _ = read_h5(mask_file, 'mask')
        vel[mask == False] = np.nan

    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(out_file, width, nlines, 1, gdal.GDT_Float32)

    dataset.SetGeoTransform([min_lon, lon_step, 0, max_lat, 0, lat_step])

    sr = osr.SpatialReference()
    sr.SetWellKnownGeogCS('WGS84')

    dataset.SetProjection(sr.ExportToWkt())
    dataset.GetRasterBand(1).WriteArray(vel)
    print('Writing data to {}.'.format(out_file))
    dataset.FlushCache()
    dataset = None
    print('done.')

## read velocity

In [ ]:
os.chdir('/ly/MintPy-Process/geo')

vel_file = 'geo_velocity.h5'
mask_file = 'geo_maskTempCoh.h5'

read_vel(vel_file, mask_file=mask_file, ds_factor=1, out_vel_file='vel.txt')

In [ ]:
!clip_vel_ts.py vel.txt v cut.kmz

In [ ]:
!make_kmz_vel.py vel_clip.txt vel_clip.kmz -v -100 100 -c jet

## velocity2geotiff

In [ ]:
os.chdir('/ly/MintPy-Process/geo')
vel2geotiff('geo_velocity.h5', 'geo_velocity_mask.tif', mask_file='geo_maskTempCoh.h5')

## read time-series and velocity

In [ ]:
os.chdir('/ly/MintPy-Process/geo')

ts_file = 'geo_timeseries_tropHgt_ramp_demErr.h5'
vel_file = 'geo_velocity.h5'
mask_file = 'geo_maskTempCoh.h5'

out_ts, date = read_vel_ts(ts_file, vel_file, mask_file, ds_factor=1, out_vel_file='vel.txt', out_ts_file='ts.txt')

In [ ]:
!clip_vel_ts.py ts.txt t cut.kmz

In [ ]:
!make_kmz_ts.py ts_clip.txt ts_clip.kmz -v -100 100 -c jet

## plot time-series displacement

In [ ]:
def plot_displacement(num_list, ts_data, date, aspect=0.2, figsize=(15, 7), y_lim=[-100, 100], fig_name=None):
    fig, ax = plt.subplots(figsize=figsize)

    ax.set_title('time series displacement', fontsize=40, fontweight='bold')
    ax.set_aspect(aspect)
    ax.minorticks_on()

    ax.set_xlabel('date', fontsize=30, labelpad=10)
    ax.xaxis.grid(True, which='major')
    ax.xaxis.set_tick_params(rotation=30, labelsize=15)
    ax.set_xmargin(0.02)

    ax.set_ylabel('displacrment (mm)', fontsize=30, labelpad=10)
    ax.set_ylim(y_lim[0], y_lim[1])
    ax.yaxis.grid(True, which='major')
    ax.yaxis.set_tick_params(rotation=0, labelsize=15)

    date_str = [str(d) for d in date]

    date = [datetime.datetime.strptime(d, '%Y%m%d') for d in date_str]

    for num in num_list:
        disp = ts_data[num + 1, 4:]
        ax.plot(date, disp, label=str(num), marker='o', linewidth=0, markersize=10)
    ax.set_xticks(date[::4])
    ax.set_xticklabels(date_str[::4])
    ax.set_yticks(list(range(y_lim[0], y_lim[1] + 10, 10)))
    ax.legend(loc='best', fontsize=20, ncol=2)
    fig.show()

    if not fig_name is None:
        fig.savefig(fig_name, dpi=200, bbox_inches='tight', facecolor='white')


num_list = [1000]
plot_displacement(num_list, out_ts, date, aspect=0.4, figsize=(30, 15), y_lim=[-100, 50], fig_name=None)
